## Network measures

### Local structures

**Indegree**
This is mostly a function of how Wikipedians revised the document and should largely be uniform across pages. The large values are likely pages with 'lists' of links.

**Outdegree**
This is 1st-order measure of an idea's influence.

### Mesoscale structures

**Clustering**
These look equally clustered among the topics.

**Centrality**
This reveals the distribution of sources of ideas within a field.

**Path lengths**

**Rich-club coefficient**

**Modularity**

**Controllability**
This is an nth-order measure of influence.

**Observability**
This is an nth-order measure of the inverse of influence.

**Coreness**
It seems that the more focused a topic is on a subtopic, the stronger the coreness. For example, genetics is heavily focused on DNA, and so it has high coreness. At the same time, in the field of economics, the concept of "economics" has high degree. Yet, it has low coreness because the field itself is heterogeneous, with major subfields such as "macroeconomics" and "microeconomics".

**Characteristic path length**
I'm not sure what path length reveals. Perhaps it is a measure of the heterogeneity in research? It describes how far one idea is to another, topologically. Cognitive science and earth science have ideas that are far away.

In [ ]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))
import wiki
import numpy as np
import pandas as pd
import networkx as nx

## Load networks

In [ ]:
topics = ['anatomy', 'biochemistry']
# topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
#           'genetics', 'immunology', 'molecular biology', 'chemistry', 'biophysics',
#           'energy', 'optics', 'earth science', 'geology', 'meteorology']#,
#           'philosophy of language', 'philosophy of law', 'philosophy of mind',
#           'philosophy of science', 'economics', 'accounting', 'education',
#           'linguistics', 'law', 'psychology', 'sociology', 'electronics',
#           'software engineering', 'robotics',
#           'calculus', 'geometry', 'abstract algebra',
#           'Boolean algebra', 'commutative algebra', 'group theory', 'linear algebra',
#           'number theory', 'dynamical systems and differential equations']

In [ ]:
path_saved = '/Users/harangju/Developer/data/wiki/graphs/dated/'
networks = {}
for topic in topics:
    print(topic, end=' ')
    networks[topic] = wiki.Net()
    networks[topic].load_graph(path_saved + topic + '.pickle')

In [ ]:
path_saved = '/Users/harangju/Developer/data/wiki/graphs/null-target/'
num_nulls = 2
null_targets = {}
for topic in topics:
    print(topic, end=' ')
    null_targets[topic] = []
    for i in range(num_nulls):
        network = wiki.Net()
        network.load_graph(path_saved + topic + '-null-' + str(i) + '.pickle')
        null_targets[topic].append(network)

## Run analysis

**NOTE:** Skip section if loading stats.

In [ ]:
import bct
from networkx.algorithms.cluster import clustering
from networkx.algorithms import betweenness_centrality
from networkx.convert_matrix import to_numpy_array
pd.options.display.max_rows = 12

In [ ]:
measures = {'indegree': lambda g: [x[1] for x in g.in_degree],
            'outdegree': lambda g: [x[1] for x in g.out_degree],
            'clustering': lambda g: list(clustering(g).values()),
            'centrality': lambda g: list(betweenness_centrality(g).values()),
            'path-length': lambda g: [y for x in list(nx.shortest_path_length(g))
                                      for y in list(x[1].values())],
            'char-path-length': lambda g: bct.charpath(to_numpy_array(g))[0],
            'modularity': lambda g: g.graph['modularity'],
            'coreness': lambda g: g.graph['coreness_be']}

In [ ]:
df = pd.DataFrame(columns=['topic','measure','value'])
for topic, network in networks.items():
    print(topic, end=' ')
    df = pd.concat([df] +
                   [pd.DataFrame([[topic, measure, func(network.graph)]],
                                 columns=['topic','measure','value'])
                    for measure, func in measures.items()],
                   ignore_index=True)

In [ ]:
for topic, null_networks in null_targets.items():
    print(topic, end=' ')
    for network in null_networks:
        df = pd.concat([df] + 
                       [pd.DataFrame([[topic, measure+'-null', func(network.graph)]],
                                     columns=['topic','measure','value'])
                        for measure, func in measures.items()],
                       ignore_index=True)

In [ ]:
df

In [ ]:
%time df_expand = df.value\
              .apply(pd.Series)\
              .merge(df, left_index=True, right_index=True)\
              .drop('value', axis=1)\
              .melt(id_vars=['topic','measure'])\
              .drop('variable', axis=1)\
              .dropna()
df_expand

In [ ]:
import pickle
pickle.dump((df, df_expand),
            open('/Users/harangju/Developer/data/wiki/analysis/stats.pickle','wb'))

## Load stats

In [ ]:
import pickle
df, df_expand = pickle.load(
    open('/Users/harangju/Developer/data/wiki/analysis/stats.pickle', 'rb'))

### Display summary

In [ ]:
df

In [ ]:
pd.unique(df.topic)

## Plot

* nice plots [seaborn](https://seaborn.pydata.org/examples/index.html)
* interactive [Bokeh](https://bokeh.pydata.org/en/latest/docs/gallery.html#gallery)

In [ ]:
# import cufflinks as cf
# cf.go_offline()
from ipywidgets import interact, widgets, Layout
import plotly
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from IPython.display import display

In [ ]:
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='whitegrid')

In [ ]:
pd.options.display.max_rows = 12
plt.rcParams.update({'figure.max_open_warning': 0})

In [ ]:
for stat in ['indegree', 'outdegree', 'clustering', 'centrality', 'path-length']:
    fig = px.box(df_expand[(df_expand.measure==stat) | (df_expand.measure==stat+'-null')],
                 x='topic', y='value', color='measure')
    fig.update_layout(template='plotly_white')
    fig.show()

In [ ]:
for stat in ['coreness', 'modularity', 'char-path-length']:
    fig = px.scatter(df_expand[(df_expand.measure==stat) |\
                               (df_expand.measure==stat+'-null')],
                     x='topic', y='value', color='measure')
    fig.update_layout(template='plotly_white')
    fig.show()

In [ ]:
df_expand

In [ ]:
data = df_expand\
        .groupby(['topic', 'measure'], as_index=False)\
        .mean()\
        .pivot(index='topic', columns='measure', values='value')\
        .reset_index()
data

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['coreness-null'],
                         y=data['coreness'],
                         mode='markers',
                         name='coreness'))
fig.add_trace(go.Scatter(x=data['modularity-null'],
                         y=data['modularity'],
                         mode='markers',
                         name='modularity'))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1],
                         mode='lines',
                         line=dict(dash='dash'),
                         name='1:1'))
fig.update_layout(template='plotly_white',
                  width=500, height=500,
                  xaxis=dict(title='null',
                             range=[0,1]),
                  yaxis=dict(title='real',
                             range=[0,1],
                             scaleanchor='x',
                             scaleratio=1))
fig.show()

### Measures in growing networks

In [ ]:
comm_t = pd.DataFrame()
for topic, network in networks.items():
    print(topic, end=' ')
    comm_t = pd.concat([comm_t] +
                       [pd.DataFrame([[topic,
                                       node,
                                       network.graph.nodes[node]['year'],
                                       network.graph.nodes[node]['community'],
                                       network.graph.nodes[node]['core_be'],
                                       network.graph.nodes[node]['core_rb'],
                                       1]],
                                     columns=['topic','node','year',
                                              'comm','core_be','core_rb',
                                              'count'])
                        for node in network.graph.nodes],
                       ignore_index=True)
comm_t = comm_t.merge(comm_t.groupby(['topic','comm'])['count'].sum(),
                      on=['topic','comm'],
                      suffixes=('','_topic_comm'))\
               .merge(comm_t.groupby(['topic','core_be'])['count'].sum(),
                      on=['topic','core_be'],
                      suffixes=('','_topic_core_be'))\
               .sort_values(by=['topic','year'])\
               .reset_index(drop=True)
comm_t['comm_count'] = comm_t.groupby(['topic','comm'])['count']\
                             .transform(pd.Series.cumsum)
comm_t['core_be_count'] = comm_t.groupby(['topic','core_be'])['count']\
                                .transform(pd.Series.cumsum)
comm_t['comm_frac'] = comm_t['comm_count']/comm_t['count_topic_comm']
comm_t['core_be_frac'] = comm_t['core_be_count']/comm_t['count_topic_core_be']
comm_t = comm_t.drop(['count','count_topic_comm','count_topic_core_be'], axis=1)

In [ ]:
comm_t

### Growth in core-periphery & modules

In [ ]:
for topic in networks.keys():
    fig = px.line(comm_t[comm_t.topic==topic],
                     x='year', y='comm_count', color='comm')
    fig.update_layout(template='plotly_white',
                      title_text=topic,
                      xaxis={'range': [0,2000]})
    fig.show()

In [ ]:
for topic in networks.keys():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=comm_t[(comm_t.topic==topic) &\
                                      (comm_t.core_be==0)]['year'],
                             y=comm_t[(comm_t.topic==topic) &\
                                      (comm_t.core_be==0)]['core_be_count'],
                             name='core_be = 0'))
    fig.add_trace(go.Scatter(x=comm_t[(comm_t.topic==topic) &\
                                      (comm_t.core_be==1)]['year'],
                             y=comm_t[(comm_t.topic==topic) &\
                                      (comm_t.core_be==1)]['core_be_count'],
                             name='core_be = 1'))
    fig.update_layout(template='plotly_white',
                      title_text=topic,
                      xaxis={'range': [0,2020]})
    fig.show()

### Birth: core vs. periphery

In [ ]:
birth = pd.concat([pd.DataFrame([[comm_t.iloc[i].topic,
                                  comm_t.iloc[i].node,
                                  comm_t.iloc[i].year,
                                  [c for c in 
                                   list(networks[comm_t.iloc[i].topic]\
                                        .graph.successors(comm_t.iloc[i].node)) + 
                                   list(networks[comm_t.iloc[i].topic]\
                                        .graph.predecessors(comm_t.iloc[i].node))
                                   if networks[comm_t.iloc[i].topic].graph.nodes[c]['core_be']]
                                 ]],
                                columns=['topic','periphery','year','cores'])
                   for i in range(len(comm_t.index))
                   if not comm_t.iloc[i].core_be],
                  ignore_index=True)
birth

In [ ]:
birth_exp = birth.cores.apply(pd.Series)\
                 .merge(birth, left_index=True, right_index=True)\
                 .drop(['cores'], axis=1)\
                 .melt(id_vars=['topic','periphery','year'], value_name='core')\
                 .drop('variable', axis=1)\
                 .dropna()\
                 .sort_values(by=['topic','year', 'periphery'])\
                 .reset_index(drop=True)
birth_exp['core_year'] = [networks[birth_exp.iloc[i].topic].graph\
                          .nodes[birth_exp.iloc[i].core]['year']
                          for i in range(len(birth_exp.index))]
birth_exp['centrality'] = [df[df.topic==birth_exp.iloc[i].topic]\
                           [df.measure=='centrality'].value.values[0]\
                           [networks[birth_exp.iloc[i].topic].nodes.index(birth_exp.iloc[i].core)]
                           for i in range(len(birth_exp.index))]
birth_exp['indegree'] = [df[df.topic==birth_exp.iloc[i].topic]\
                           [df.measure=='indegree'].value.values[0]\
                           [networks[birth_exp.iloc[i].topic].nodes.index(birth_exp.iloc[i].core)]
                           for i in range(len(birth_exp.index))]
birth_exp['outdegree'] = [df[df.topic==birth_exp.iloc[i].topic]\
                           [df.measure=='outdegree'].value.values[0]\
                           [networks[birth_exp.iloc[i].topic].nodes.index(birth_exp.iloc[i].core)]
                           for i in range(len(birth_exp.index))]

In [ ]:
birth_exp

In [ ]:
for topic in networks.keys():
    fig = px.scatter(birth_exp[birth_exp.topic==topic],
                     x='year', y='core_year', color='outdegree')
    fig.update_layout(template='plotly_white',
                      title_text=topic,
                      width=500, height=500,
                      yaxis={'scaleanchor': 'x',
                             'scaleratio': 1})
    fig.show()

### Cores in communities

In [ ]:
comm_core = pd.concat([pd.DataFrame([[node,
                                      graph.nodes[node]['year'],
                                      graph.nodes[node]['community'],
                                      graph.nodes[node]['community_core_be'],
                                      1 if graph.nodes[node]['community_core_be']==0 else 0,
                                      graph.graph['community_coreness_be']\
                                          [graph.nodes[node]['community']],
                                      1
                                     ]],
                                    columns=['node','year','community','community_core',
                                             'community_peri','community_coreness','count'])
                       for node in graph.nodes],
                      ignore_index=True)\
              .sort_values(by='year')
comm_core = comm_core\
              .merge(comm_core.groupby(['community'])['count'].sum(),
                     on=['community'],
                     suffixes=('','_sum'))
comm_core['core_count'] = comm_core.groupby(['community'])['community_core']\
                                             .transform(pd.Series.cumsum)
comm_core['peri_count'] = comm_core.groupby(['community'])['community_peri']\
                                             .transform(pd.Series.cumsum)
comm_core = comm_core.drop(['count', 'count_sum', 'community_core', 'community_peri'], axis=1)
comm_core

In [ ]:
for i in range(5):#range(max([graph.nodes[n]['community'] for n in graph.nodes]) + 1):
    fig = go.Figure()
    data = comm_core[comm_core.community==i]
    fig.add_trace(go.Scatter(x=data['year'],
                             y=data['core_count'],
                             mode='lines',
                             name='# cores'))
    fig.add_trace(go.Scatter(x=data['year'],
                             y=data['peri_count'],
                             mode='lines',
                             name='# periphery'))
    fig.update_layout(template='plotly_white',
                      title_text=f"community {i+1}",
                      height=400)
    fig.show()

### i